# Experimental notebook

This is intended for prodding and poking the data structures, not for actual reporting or serious use.
It should definitely, 100% never be trusted for doing anything serious.

In [8]:
import tdt
import collections
import numpy as np
import sys:
import seaborn as sns
import matplotlib.pylab as plt
# help(tdt.read_block)


SyntaxError: invalid syntax (<ipython-input-8-a38a3c2c4d71>, line 4)

In [9]:
TONE_DURATION = 0.25
IN_TONE_CAPTURE_START_OFFSET = 0.0
IN_TONE_CAPTURE_END_OFFSET = 0.2  # time between start offset and end offset will be counted
OUT_TONE_CAPTURE_START_OFFSET = 0.1
OUT_TONE_CAPTURE_END_OFFSET = 0.25 # time between start offset and end offset will be counted

TONES_PER_SECOND = 2

START_OFFSET = -9.1
END_OFFSET = 9.1

BLOCK_PATH = "/Users/smorgan/Documents/tdttmp/Block-1"
# BLOCK_PATH = "/Volumes/SM Research/Research Data/Simeon Morgan/111_140/2009-07-19 Auditory continuous 10s acclimatisation/2009-07-19 111_140/Block-1"

In [10]:
data = tdt.read_block(BLOCK_PATH, evtype=['epocs', 'snips', 'scalars'], nodata=1)

/Volumes/CaseSensitiveDevPartition/personal/tdt/.venv/lib/python3.8/site-packages/tdt/TDTbin2py.py:792: Warning: tnt file could not be processed
  warnings.warn('tnt file could not be processed', Warning)


read from t=0s to t=7457.32s


In [11]:
filtered = tdt.epoc_filter(data, "TriS", t=[START_OFFSET, END_OFFSET - START_OFFSET])

In [6]:
# Skip the last tone as the end offset should always go a little past the end
tones_to_include = int(((END_OFFSET - START_OFFSET) * TONES_PER_SECOND) - 1)

TIME_RANGE_ONSET_IDX = 0
TIME_RANGE_OFFSET_IDX = 1

SpikeCountRange = collections.namedtuple("SpikeCountRange", ("start", "end"))

all_trials_data = []
cspk_offset = 0
cspk_data = filtered["snips"]["CSPK"]
cspk_length = len(cspk_data["ts"])
np.set_printoptions(threshold=sys.maxsize)

for trial_number in range(len(filtered["time_ranges"][0])):
    in_tone_data = np.zeros((tones_to_include * 2, 32), np.uint)
    #out_tone_data = np.zeros((tones_to_include, 32), np.uint)
    for tone_number, tone_offset in enumerate(filter(lambda x: filtered["time_ranges"][TIME_RANGE_ONSET_IDX][trial_number] < x < filtered["time_ranges"][TIME_RANGE_OFFSET_IDX][trial_number], filtered["epocs"]["StiS"]["onset"])):
        if tone_number >= tones_to_include:
            continue

        spike_cumulator_range = SpikeCountRange(start=tone_offset+IN_TONE_CAPTURE_START_OFFSET, end=tone_offset+IN_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1
            
        spike_cumulator_range = SpikeCountRange(start=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_START_OFFSET, end=tone_offset+TONE_DURATION+OUT_TONE_CAPTURE_END_OFFSET)
        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.start:
            cspk_offset += 1

        while cspk_data["ts"][cspk_offset] < spike_cumulator_range.end:
            in_tone_data[tone_number * 2 + 1, cspk_data["chan"][cspk_offset][0]-1] += 1
            cspk_offset += 1


#     print(in_tone_data)
#     plt.rcParams['figure.figsize'] = [100/2.54, 80/2.54]
#     plt.imshow(in_tone_data, cmap='hot', interpolation='nearest')
#     plt.show()
#     ax = sns.heatmap(in_tone_data, linewidth=0.5)
#     plt.show()
#     print("Out tone data")
#     print(out_tone_data)

In [15]:
filtered["epocs"]["StiS"]["offset"][0] - filtered["epocs"]["StiS"]["onset"][0]

0.2500198399999931

In [1]:
filtered["epocs"]

NameError: name 'filtered' is not defined

In [16]:
start, end = list(zip(filtered["time_ranges"][0], filtered["time_ranges"][1]))[0]

[(112.58380288000001, 112.83382272, 192.0),
 (113.08376064000001, 113.33382144000001, 193.0),
 (113.58380032000001, 113.83386112000001, 194.0),
 (114.08375808, 114.33381888, 195.0),
 (114.5837568, 114.8338176, 196.0),
 (115.08371456, 115.33381632, 197.0),
 (115.58375424, 115.83381504, 198.0),
 (116.083712, 116.3337728, 199.0),
 (116.58371072, 116.83381248, 200.0),
 (117.08370944, 117.3338112, 201.0),
 (117.58383104, 117.83380992000001, 202.0),
 (118.08370688000001, 118.33376768000001, 203.0),
 (118.5837056, 118.8337664, 204.0),
 (119.08370432, 119.33376512000001, 205.0),
 (119.58370304, 119.83376384, 206.0),
 (120.08370176, 120.33376256, 207.0),
 (120.58370048, 120.83376128, 208.0),
 (121.0836992, 121.33376, 209.0),
 (121.58369792, 121.83379968, 210.0),
 (122.08365568, 122.33375744, 211.0),
 (122.5836544, 122.83375616000001, 212.0),
 (123.08365312000001, 123.33375488, 213.0),
 (123.58365184, 123.83371264, 214.0),
 (124.08365056000001, 124.33371136000001, 215.0),
 (124.58364928, 124.833